In [1]:
import netCDF4 as nc
import numpy as np

In [2]:
dataset_path = "../UFZ_RemoteSensing/HOLAPS-LE-JJA_anomaly-d-2001-2005.nc"

long_range = [100,200]
lat_range = [50,70]
time_range = [0,50]

### Get input
Notice the flipped order of long and lat

In [3]:
ds = nc.Dataset(dataset_path)

variable_names = list(ds.variables.keys())
print(variable_names)

['time', 'longitude', 'latitude', 'surface_upward_latent_heat_flux']


In [6]:
for n in variable_names:
    print(n, ds[n].shape)

time (460,)
longitude (1233,)
latitude (601,)
surface_upward_latent_heat_flux (460, 601, 1233)


### Create output

In [149]:
## create output nc file:
ncfile = nc.Dataset('tiny.nc',mode='w',format='NETCDF4_CLASSIC') 

time_dim = ncfile.createDimension(variable_names[0], time_range[1]-time_range[0]) 
#time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).
lon_dim = ncfile.createDimension(variable_names[1], long_range[1]-long_range[0])    # longitude axis
lat_dim = ncfile.createDimension(variable_names[2], lat_range[1]-lat_range[0])     # latitude axis

for dim in ncfile.dimensions.items():
    print(dim)

('time', <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 50)
('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 100)
('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 20)


In [150]:
ncfile.title='test'
print(ncfile.title)

test


In [151]:

# Define two variables with the same names as dimensions,
# a conventional way to define "coordinate variables".
time = ncfile.createVariable(variable_names[0], np.float64, (variable_names[0],))
time.units = 'day as %Y%m%d.%f'
time.long_name = variable_names[0]

lon = ncfile.createVariable(variable_names[1], np.float32, (variable_names[1],))
lon.units = 'degrees_east'
lon.long_name = variable_names[1]

lat = ncfile.createVariable(variable_names[2], np.float32, (variable_names[2],))
lat.units = 'degrees_north'
lat.long_name = variable_names[2]

# Define a 3D variable to hold the data
heat = ncfile.createVariable(variable_names[3],np.float64,
                             (variable_names[0],variable_names[2],variable_names[1])) 
# note: unlimited dimension is leftmost
heat.units = 'W m-2'
heat.standard_name = 'heat' # this is a CF standard name

In [152]:
# fill in data
time[:] = ds[variable_names[0]][time_range[0]:time_range[1]]
lon[:] = ds[variable_names[1]][long_range[0]:long_range[1]]
lat[:] = ds[variable_names[2]][lat_range[0]:lat_range[1]]
heat[:] = ds[variable_names[3]][time_range[0]:time_range[1], lat_range[0]:lat_range[1], long_range[0]:long_range[1]]

heat[3] = heat[0,0,0]


In [153]:
ncfile.close(); print('Dataset is closed!')

Dataset is closed!
